In [9]:
import cv2
import os
import numpy as np
import polars as pl
from IPython.display import display

In [10]:
data_path = "data"
image_data_path = os.path.join(data_path, "image")
generated_image_data_path = os.path.join(image_data_path, "generated")

In [11]:
image_classes = os.listdir(generated_image_data_path)
image_paths = {image_class: [] for image_class in image_classes}
for image_class in image_classes:
    image_path = os.path.join(generated_image_data_path, image_class)
    image_paths[image_class] = sorted(os.listdir(image_path))

In [12]:
images = {image_class: [] for image_class in image_classes}
for image_class, image_list in image_paths.items():
    images[image_class] = [
        cv2.imread(
            os.path.join(
                generated_image_data_path,
                image_class,
                image_name,
            )
        )
        for image_name in image_list
    ]

In [13]:
# noinspection PyDictCreation
def extract_features(image):
    blue  = image[:, :, 0]
    green = image[:, :, 1]
    red   = image[:, :, 2]

    features = {}

    features["blue_mean"] = float(np.mean(blue))
    features["blue_std"] = float(np.std(blue))
    features["green_mean"] = float(np.mean(green))
    features["green_std"] = float(np.std(green))
    features["red_mean"] = float(np.mean(red))
    features["red_std"] = float(np.std(red))

    small = cv2.resize(image, (16, 16), interpolation=cv2.INTER_AREA)
    gray_small = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)
    gray_flat = gray_small.reshape(-1).astype("float32")
    for i, val in enumerate(gray_flat):
        features[f"gray_{i:03d}"] = float(val) # type: ignore

    return features

In [14]:
rows = [
    {**extract_features(img), "class": img_class}
    for img_class, img_matrices in images.items()
    for img in img_matrices
]
df = pl.DataFrame(rows)
df.write_csv(os.path.join(data_path, "tabular", "feature_extraction.csv"))
display(df)

blue_mean,blue_std,green_mean,green_std,red_mean,red_std,gray_000,gray_001,gray_002,gray_003,gray_004,gray_005,gray_006,gray_007,gray_008,gray_009,gray_010,gray_011,gray_012,gray_013,gray_014,gray_015,gray_016,gray_017,gray_018,gray_019,gray_020,gray_021,gray_022,gray_023,gray_024,gray_025,gray_026,gray_027,gray_028,gray_029,gray_030,…,gray_220,gray_221,gray_222,gray_223,gray_224,gray_225,gray_226,gray_227,gray_228,gray_229,gray_230,gray_231,gray_232,gray_233,gray_234,gray_235,gray_236,gray_237,gray_238,gray_239,gray_240,gray_241,gray_242,gray_243,gray_244,gray_245,gray_246,gray_247,gray_248,gray_249,gray_250,gray_251,gray_252,gray_253,gray_254,gray_255,class
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
75.192955,95.64547,92.357155,105.600454,99.321049,112.099186,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,…,236.0,239.0,231.0,236.0,230.0,234.0,233.0,227.0,226.0,230.0,235.0,237.0,236.0,235.0,234.0,236.0,237.0,235.0,234.0,234.0,226.0,230.0,233.0,235.0,230.0,231.0,233.0,236.0,234.0,234.0,237.0,237.0,237.0,238.0,237.0,236.0,"""banana"""
169.612247,75.303959,187.843922,51.601779,194.192684,47.84109,164.0,167.0,170.0,174.0,178.0,183.0,187.0,194.0,200.0,201.0,203.0,206.0,209.0,213.0,214.0,216.0,172.0,174.0,179.0,181.0,181.0,182.0,185.0,193.0,198.0,201.0,205.0,211.0,215.0,217.0,218.0,…,214.0,214.0,216.0,218.0,158.0,169.0,170.0,152.0,118.0,126.0,160.0,172.0,178.0,185.0,205.0,210.0,209.0,208.0,210.0,214.0,158.0,166.0,169.0,171.0,173.0,167.0,173.0,179.0,192.0,201.0,206.0,205.0,206.0,204.0,206.0,209.0,"""banana"""
157.572319,73.423733,193.686344,48.08909,210.02092,47.674383,43.0,41.0,36.0,27.0,20.0,29.0,59.0,93.0,130.0,167.0,187.0,193.0,199.0,201.0,208.0,205.0,40.0,68.0,102.0,137.0,174.0,193.0,197.0,198.0,202.0,204.0,204.0,206.0,208.0,210.0,214.0,…,232.0,231.0,234.0,235.0,207.0,211.0,214.0,221.0,227.0,230.0,229.0,232.0,230.0,230.0,228.0,232.0,238.0,243.0,228.0,205.0,229.0,230.0,232.0,230.0,234.0,230.0,231.0,233.0,230.0,233.0,240.0,237.0,219.0,192.0,174.0,166.0,"""banana"""
150.701717,65.034955,179.173409,59.486491,182.290588,76.773148,82.0,67.0,57.0,54.0,50.0,52.0,57.0,69.0,76.0,88.0,97.0,99.0,98.0,104.0,104.0,111.0,73.0,52.0,38.0,72.0,148.0,198.0,223.0,229.0,215.0,180.0,128.0,91.0,101.0,107.0,107.0,…,207.0,143.0,135.0,145.0,204.0,205.0,173.0,224.0,240.0,240.0,237.0,234.0,231.0,228.0,226.0,187.0,140.0,139.0,142.0,147.0,109.0,108.0,90.0,97.0,157.0,196.0,213.0,213.0,197.0,167.0,132.0,128.0,134.0,138.0,144.0,150.0,"""banana"""
51.750942,52.189217,111.73209,77.884898,154.641254,90.336133,3.0,3.0,4.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,6.0,24.0,44.0,61.0,4.0,4.0,4.0,3.0,3.0,2.0,3.0,3.0,4.0,18.0,39.0,59.0,74.0,67.0,55.0,…,197.0,198.0,201.0,200.0,45.0,48.0,58.0,72.0,89.0,116.0,149.0,177.0,191.0,195.0,194.0,192.0,190.0,187.0,182.0,184.0,82.0,96.0,115.0,139.0,159.0,173.0,181.0,183.0,181.0,181.0,177.0,175.0,172.0,182.0,195.0,195.0,"""banana"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
84.33025,55.283538,140.394249,72.3681,196.035351,68.273515,70.0,71.0,73.0,74.0,75.0,75.0,75.0,74.0,72.0,70.0,67.0,64.0,61.0,58.0,55.0,52.0,71.0,72.0,73.0,73.0,74.0,75.0,74.0,73.0,71.0,69.0,67.0,63.0,60.0,57.0,53.0,…,207.0,207.0,207.0,207.0,188.0,190.0,192.0,193.0,193.0,194.0,195.0,195.0,197.0,197.0,197.0,197.0,197.0,198.0,198.0,198.0,173.0,175.0,176.0,178.0,179.0,180.0,181.0,181.0,182.0,182.0,183.0,183.0,183.0,183.0,184.0,184.0,"""tomato"""
104.729927,72.279619,131.530888,69.865022,194.016422,33.864521,170.0,170.0,171.0,171.0,171.0,171.0,170.0,170.0,170.0,170.0,170.0,170.0,170.0,169.0,169.0,169.0,170.0,171.0,171.0,171.0,171.0,171.0,171